In [8]:
import re
from os import listdir
from collections import Counter
import pandas as pd

In [26]:
mypath = '../luxoft/source/'
files = listdir(mypath)
words = []

for i in files:
    f = open(mypath + i, 'r', encoding='utf-8')
    t = f.read().replace('\n', ' ')
    t = t.split(' ')
    for i in t:
        i = re.sub('\W', ' ', i)
        i = i.split(' ')
        for j in i:
            words.append(j)
    f.close()

sorted(Counter(words).items(), key=lambda x: x[1], reverse=True)[:5]

[('', 46412),
 ('Senior', 9457),
 ('Developer', 7706),
 ('Engineer', 5997),
 ('India', 5803)]

In [28]:
mypath = '../luxoft/source/'
files = listdir(mypath)

for file in files:
    keywords = []
    headers = pd.read_csv(mypath + file, header=None)[1].to_list()
    for i in headers:
        i = re.sub('[^A-z+#]', ' ', i)
        i = re.sub('Data\s', 'Data_', i)
        i = re.sub('\sMode', '_Mode', i)
        i = re.sub('\sManager', '_Manager', i)
        i = re.sub('\sManagement', '_Management', i)
        i = re.sub('\sAnalyst', '_Analyst', i)
        i = re.sub('\sspeaker', '_speaker', i)
        i = re.sub('\sLearning', '_Learning', i)
        i = re.sub('\send', '_end', i)
        i = re.sub('\sStack', '_Stack', i)
        i = re.sub('\sLead', '_Lead', i)
        i = i.split(' ')
        for j in i:
            if len(j) > 1 and j not in ('with', 'and', 'IT', 'to', 'in', 'for', 'or', 'experience', 'of', 'Project'):
                keywords.append(j)
                if j == 'Mode':
                    print(i)

['Embedded', 'Kernel', 'Mode', 'Driver', 'Developer']
['Embedded', 'Kernel', 'Mode', 'Driver', 'Developer']
['Embedded', 'Kernel', 'Mode', 'Driver', 'Developer']
['Embedded', 'Kernel', 'Mode', 'Driver', 'Developer']
['Windows', 'Linux', 'Kernel', 'Mode', 'Drivers', 'Developer']
['Windows', 'Linux', 'Kernel', 'Mode', 'Drivers', 'Developer']
['Windows', 'Linux', 'Kernel', 'Mode', 'Drivers', 'Developer']
['Windows', 'Linux', 'Kernel', 'Mode', 'Drivers', 'Developer']
['Windows', 'Linux', 'Kernel', 'Mode', 'Drivers', 'Developer']
['Windows', 'Linux', 'Kernel', 'Mode', 'Drivers', 'Developer']
['Windows', 'Linux', 'Kernel', 'Mode', 'Drivers', 'Developer']
['Windows', 'Linux', 'Kernel', 'Mode', 'Drivers', 'Developer']
['Windows', 'Linux', 'Kernel', 'Mode', 'Drivers', 'Developer']
['Windows', 'Linux', 'Kernel', 'Mode', 'Drivers', 'Developer']
['Windows', 'Linux', 'Kernel', 'Mode', 'Drivers', 'Developer']
['Windows', 'Linux', 'Kernel', 'Mode', 'Drivers', 'Developer']
['Windows', 'Linux', 'Kernel